# Exercise 2: Advanced Analytics NLP

In [1]:
!pip install spark-nlp==1.7.3

    100% |████████████████████████████████| 72.8MB 496kB/s eta 0:00:01  1% |▌                               | 1.1MB 21.1MB/s eta 0:00:04    2% |█                               | 2.1MB 17.1MB/s eta 0:00:05    4% |█▍                              | 3.1MB 23.9MB/s eta 0:00:03    5% |█▉                              | 4.2MB 20.2MB/s eta 0:00:04    7% |██▎                             | 5.3MB 23.1MB/s eta 0:00:03    10% |███▎                            | 7.4MB 21.6MB/s eta 0:00:04    14% |████▋                           | 10.6MB 22.5MB/s eta 0:00:03    17% |█████▋                          | 12.8MB 21.9MB/s eta 0:00:03    19% |██████                          | 13.9MB 21.9MB/s eta 0:00:03    23% |███████▌                        | 17.1MB 21.9MB/s eta 0:00:03    24% |████████                        | 18.2MB 23.2MB/s eta 0:00:03    26% |████████▌                       | 19.3MB 24.2MB/s eta 0:00:03    29% |█████████▍                      | 21.4MB 23.5MB/s eta 0:00:03    30% |██████████              

In [2]:
import pandas as pd
pd.set_option('max_colwidth', 800)

# Create a spark context that includes a 3rd party jar for NLP

In [3]:
#jarPath = "spark-nlp-assembly-1.7.3.jar"

from pyspark.sql import SparkSession

spark = SparkSession.builder.config("spark.jars.packages","JohnSnowLabs:spark-nlp:1.8.2") \
    .getOrCreate()

# Read multiple files in a dir as one Dataframe

In [4]:
dataPath = "./data/reddit/*.json"
df = spark.read.json(dataPath)
print(df.count())
df.printSchema()

100
root
 |-- data: struct (nullable = true)
 |    |-- approved_at_utc: string (nullable = true)
 |    |-- approved_by: string (nullable = true)
 |    |-- archived: boolean (nullable = true)
 |    |-- author: string (nullable = true)
 |    |-- author_flair_background_color: string (nullable = true)
 |    |-- author_flair_css_class: string (nullable = true)
 |    |-- author_flair_richtext: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- author_flair_template_id: string (nullable = true)
 |    |-- author_flair_text: string (nullable = true)
 |    |-- author_flair_text_color: string (nullable = true)
 |    |-- author_flair_type: string (nullable = true)
 |    |-- author_fullname: string (nullable = true)
 |    |-- author_patreon_flair: boolean (nullable = true)
 |    |-- banned_at_utc: string (nullable = true)
 |    |-- banned_by: string (nullable = true)
 |    |-- can_gild: boolean (nullable = true)
 |    |-- can_mod_post: boolean (nullable = true)


# Deal with Struct type to query subfields 

In [6]:
title = "data.title"
author = "data.author"

dfAuthor = df.select(title,author)

# Try to implement the equivalent of flatMap in dataframes

In [14]:
import pyspark.sql.functions as F

dfWordCount = df.select(F.explode(F.split(title," "))).groupby("col").count().orderBy(F.desc("count"))

In [15]:
dfWordCount.limit(5).toPandas()

,col,count
0,to,58
1,the,46
2,of,42
3,in,41
4,a,25


# Use an NLP libary to do Part-of-Speech Tagging

In [18]:
from com.johnsnowlabs.nlp.pretrained.pipeline.en import BasicPipeline as bp
dfAnnotated = bp.annotate(dfAuthor,"title")
dfAnnotated.printSchema()

root
 |-- text: string (nullable = true)
 |-- author: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |-- token: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |-- normal: array (nullable = true)
 |    |-- element: struct (contains

In [20]:
dfAnnotated.limit(2).toPandas()

,text,author,document,token,normal,lemma,pos
0,"Microsoft Corp said it has discovered hacking targeting democratic institutions, think tanks, and non-profit organizations in Europe.",jaykirsch,"[(document, 0, 132, Microsoft Corp said it has discovered hacking targeting democratic institutions, think tanks, and non-profit organizations in Europe., {})]","[(token, 0, 8, Microsoft, {'sentence': '1'}), (token, 10, 13, Corp, {'sentence': '1'}), (token, 15, 18, said, {'sentence': '1'}), (token, 20, 21, it, {'sentence': '1'}), (token, 23, 25, has, {'sentence': '1'}), (token, 27, 36, discovered, {'sentence': '1'}), (token, 38, 44, hacking, {'sentence': '1'}), (token, 46, 54, targeting, {'sentence': '1'}), (token, 56, 65, democratic, {'sentence': '1'}), (token, 67, 78, institutions, {'sentence': '1'}), (token, 79, 79, ,, {'sentence': '1'}), (token, 81, 85, think, {'sentence': '1'}), (token, 87, 91, tanks, {'sentence': '1'}), (token, 92, 92, ,, {'sentence': '1'}), (token, 94, 96, and, {'sentence': '1'}), (token, 98, 107, non-profit, {'sentence': '1'}), (token, 109, 121, organizations, {'sentence': '1'}), (token, 123, 124, in, {'sentence': '1'})...","[(token, 0, 8, Microsoft, {'sentence': '1'}), (token, 10, 13, Corp, {'sentence': '1'}), (token, 15, 18, said, {'sentence': '1'}), (token, 20, 21, it, {'sentence': '1'}), (token, 23, 25, has, {'sentence': '1'}), (token, 27, 36, discovered, {'sentence': '1'}), (token, 38, 44, hacking, {'sentence': '1'}), (token, 46, 54, targeting, {'sentence': '1'}), (token, 56, 65, democratic, {'sentence': '1'}), (token, 67, 78, institutions, {'sentence': '1'}), (token, 81, 85, think, {'sentence': '1'}), (token, 87, 91, tanks, {'sentence': '1'}), (token, 94, 96, and, {'sentence': '1'}), (token, 98, 107, nonprofit, {'sentence': '1'}), (token, 109, 121, organizations, {'sentence': '1'}), (token, 123, 124, in, {'sentence': '1'}), (token, 126, 131, Europe, {'sentence': '1'})]","[(token, 0, 8, Microsoft, {'sentence': '1'}), (token, 10, 13, Corp, {'sentence': '1'}), (token, 15, 18, say, {'sentence': '1'}), (token, 20, 21, it, {'sentence': '1'}), (token, 23, 25, have, {'sentence': '1'}), (token, 27, 36, discover, {'sentence': '1'}), (token, 38, 44, hack, {'sentence': '1'}), (token, 46, 54, target, {'sentence': '1'}), (token, 56, 65, democratic, {'sentence': '1'}), (token, 67, 78, institution, {'sentence': '1'}), (token, 79, 79, ,, {'sentence': '1'}), (token, 81, 85, think, {'sentence': '1'}), (token, 87, 91, tank, {'sentence': '1'}), (token, 92, 92, ,, {'sentence': '1'}), (token, 94, 96, and, {'sentence': '1'}), (token, 98, 107, non-profit, {'sentence': '1'}), (token, 109, 121, organization, {'sentence': '1'}), (token, 123, 124, in, {'sentence': '1'}), (token, 1...","[(pos, 0, 8, NNP, {'word': 'Microsoft'}), (pos, 10, 13, NNP, {'word': 'Corp'}), (pos, 15, 18, VBD, {'word': 'said'}), (pos, 20, 21, PRP, {'word': 'it'}), (pos, 23, 25, VBZ, {'word': 'has'}), (pos, 27, 36, VBN, {'word': 'discovered'}), (pos, 38, 44, VBG, {'word': 'hacking'}), (pos, 46, 54, VBG, {'word': 'targeting'}), (pos, 56, 65, JJ, {'word': 'democratic'}), (pos, 67, 78, NNS, {'word': 'institutions'}), (pos, 81, 85, VBP, {'word': 'think'}), (pos, 87, 91, NNS, {'word': 'tanks'}), (pos, 94, 96, CC, {'word': 'and'}), (pos, 98, 107, NN, {'word': 'nonprofit'}), (pos, 109, 121, NNS, {'word': 'organizations'}), (pos, 123, 124, IN, {'word': 'in'}), (pos, 126, 131, NNP, {'word': 'Europe'})]"
1,Deutsche Bank reportedly planned to extend the dates of $340 million in loans to Trump Organization to avoid a potential nightmare of chasing a sitting president for cash,canuck_burger,"[(document, 0, 169, Deutsche Bank reportedly planned to extend the dates of $340 million in loans to Trump Organization to avoid a potential nightmare of chasing a sitting president for cash, {})]","[(token, 0, 7, Deutsche, {'sentence': '1'}), (token, 9, 12, Bank, {'sentence': '1'}), (token, 14, 23, reportedly, {'sentence': '1'}), (token, 25, 31, planned, {'sentence': '1'}), (token, 33, 3

## Deal with Map type to query subfields

In [21]:
dfPos = dfAnnotated.select("text", "pos.metadata", "pos.result")
dfPos.limit(5).toPandas()

,text,metadata,result
0,"Microsoft Corp said it has discovered hacking targeting democratic institutions, think tanks, and non-profit organizations in Europe.","[{'word': 'Microsoft'}, {'word': 'Corp'}, {'word': 'said'}, {'word': 'it'}, {'word': 'has'}, {'word': 'discovered'}, {'word': 'hacking'}, {'word': 'targeting'}, {'word': 'democratic'}, {'word': 'institutions'}, {'word': 'think'}, {'word': 'tanks'}, {'word': 'and'}, {'word': 'nonprofit'}, {'word': 'organizations'}, {'word': 'in'}, {'word': 'Europe'}]","[NNP, NNP, VBD, PRP, VBZ, VBN, VBG, VBG, JJ, NNS, VBP, NNS, CC, NN, NNS, IN, NNP]"
1,Deutsche Bank reportedly planned to extend the dates of $340 million in loans to Trump Organization to avoid a potential nightmare of chasing a sitting president for cash,"[{'word': 'Deutsche'}, {'word': 'Bank'}, {'word': 'reportedly'}, {'word': 'planned'}, {'word': 'to'}, {'word': 'extend'}, {'word': 'the'}, {'word': 'dates'}, {'word': 'of'}, {'word': 'million'}, {'word': 'in'}, {'word': 'loans'}, {'word': 'to'}, {'word': 'Trump'}, {'word': 'Organization'}, {'word': 'to'}, {'word': 'avoid'}, {'word': 'a'}, {'word': 'potential'}, {'word': 'nightmare'}, {'word': 'of'}, {'word': 'chasing'}, {'word': 'a'}, {'word': 'sitting'}, {'word': 'president'}, {'word': 'for'}, {'word': 'cash'}]","[NNP, NNP, RB, VBD, TO, VB, DT, NNS, IN, CD, IN, NNS, TO, NNP, NNP, TO, VB, DT, JJ, NN, IN, VBG, DT, VBG, NN, IN, NN]"
2,"Iranian ""morality police"" were forced to fire warning shots when a crowd intervened to prevent them from arresting two women for not wearing a hijab. The incident occurred in Tehran's northeastern Narmak neighbourhood on Friday night, and ended with a mob tearing the door off a police vehicle.","[{'word': 'Iranian'}, {'word': 'morality'}, {'word': 'police'}, {'word': 'were'}, {'word': 'forced'}, {'word': 'to'}, {'word': 'fire'}, {'word': 'warning'}, {'word': 'shots'}, {'word': 'when'}, {'word': 'a'}, {'word': 'crowd'}, {'word': 'intervened'}, {'word': 'to'}, {'word': 'prevent'}, {'word': 'them'}, {'word': 'from'}, {'word': 'arresting'}, {'word': 'two'}, {'word': 'women'}, {'word': 'for'}, {'word': 'not'}, {'word': 'wearing'}, {'word': 'a'}, {'word': 'hijab'}, {'word': 'The'}, {'word': 'incident'}, {'word': 'occurred'}, {'word': 'in'}, {'word': 'Tehran'}, {'word': 's'}, {'word': 'northeastern'}, {'word': 'Narmak'}, {'word': 'neighbourhood'}, {'word': 'on'}, {'word': 'Friday'}, {'word': 'night'}, {'word': 'and'}, {'word': 'ended'}, {'word': 'with'}, {'word': 'a'}, {'word': 'mob'...","[JJ, NN, NN, VBD, VBN, TO, VB, NN, NNS, WRB, DT, NN, VBD, TO, VB, PRP, IN, VBG, CD, NNS, IN, RB, VBG, DT, NN, DT, NN, VBD, IN, NNP, VBZ, JJ, NNP, NN, IN, NNP, NN, CC, VBD, IN, DT, NN, VBG, DT, NN, RP, DT, NN, NN]"
3,"Trump administration 'pushing Saudi nuclear deal' which could benefit company linked to Jared Kushner - Senior Trump administration officials pushed a project to share nuclear power technology with Saudi Arabia over the objections of ethics officials, according to a congressional report","[{'word': 'Trump'}, {'word': 'administration'}, {'word': 'pushing'}, {'word': 'Saudi'}, {'word': 'nuclear'}, {'word': 'deal'}, {'word': 'which'}, {'word': 'could'}, {'word': 'benefit'}, {'word': 'company'}, {'word': 'linked'}, {'word': 'to'}, {'word': 'Jared'}, {'word': 'Kushner'}, {'word': 'Senior'}, {'word': 'Trump'}, {'word': 'administration'}, {'word': 'officials'}, {'word': 'pushed'}, {'word': 'a'}, {'word': 'project'}, {'word': 'to'}, {'word': 'share'}, {'word': 'nuclear'}, {'word': 'power'}, {'word': 'technology'}, {'word': 'with'}, {'word': 'Saudi'}, {'word': 'Arabia'}, {'word': 'over'}, {'word': 'the'}, {'word': 'objections'}, {'word': 'of'}, {'word': 'ethics'}, {'word': 'officials'}, {'word': 'according'}, {'word': 'to'}, {'word': 'a'}, {'word': 'congressional'}, {'word': 're...","[NNP, NN, VBG, NNP, NN, NN, WDT, MD, VB, NN, VBN, TO, NNP, NNP, NNP, NNP, NN, NNS, VBD, DT, NN, TO, VB, JJ, NN, NN, IN, NNP, NNP, IN, DT, NNS, IN, NNS, NNS, VBG, TO, DT, JJ, NN

In [22]:
dfPos= dfAnnotated.select(F.explode("pos").alias("pos"))
dfPos.limit(10).toPandas()

,pos
0,"(pos, 0, 8, NNP, {'word': 'Microsoft'})"
1,"(pos, 10, 13, NNP, {'word': 'Corp'})"
2,"(pos, 15, 18, VBD, {'word': 'said'})"
3,"(pos, 20, 21, PRP, {'word': 'it'})"
4,"(pos, 23, 25, VBZ, {'word': 'has'})"
5,"(pos, 27, 36, VBN, {'word': 'discovered'})"
6,"(pos, 38, 44, VBG, {'word': 'hacking'})"
7,"(pos, 46, 54, VBG, {'word': 'targeting'})"
8,"(pos, 56, 65, JJ, {'word': 'democratic'})"
9,"(pos, 67, 78, NNS, {'word': 'institutions'})"


## Keep only proper nouns NNP or NNPS

In [25]:
nnpFilter = "pos.result = 'NNP' or pos.result = 'NNPS' "
dfNNP = dfPos.where(nnpFilter)
dfNNP.limit(5).toPandas()

,pos
0,"(pos, 0, 8, NNP, {'word': 'Microsoft'})"
1,"(pos, 10, 13, NNP, {'word': 'Corp'})"
2,"(pos, 126, 131, NNP, {'word': 'Europe'})"
3,"(pos, 0, 7, NNP, {'word': 'Deutsche'})"
4,"(pos, 9, 12, NNP, {'word': 'Bank'})"


In [27]:
dfNNP.printSchema()

root
 |-- pos: struct (nullable = true)
 |    |-- annotatorType: string (nullable = true)
 |    |-- begin: integer (nullable = false)
 |    |-- end: integer (nullable = false)
 |    |-- result: string (nullable = true)
 |    |-- metadata: map (nullable = true)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = true)



,value,tag
0,Microsoft,NNP
1,Corp,NNP
2,Europe,NNP
3,Deutsche,NNP
4,Bank,NNP


## Extract columns form a map in a col

In [45]:
dfWordTag = dfNNP.selectExpr("pos.metadata['word'] as value", "pos.result as tag")

In [47]:
from pyspark.sql.functions import desc
dfWordTag.groupBy("value").count().orderBy(desc("count")).limit(20).toPandas()

,value,count
0,US,14
1,Trump,9
2,Saudi,8
3,Putin,7
4,Russia,6
5,Europe,5
6,Arabia,5
7,Catholic,4
8,UK,4
9,Vladimir,4
